# NPS

В этой тетрадке мы построим доверительный интервал для NPS. 

**NPS (Net Promoter Score)** — индекс потребительской лояльности, основанный на готовности рекомендовать продукт/услугу/сервис/бренд. Работает в качестве показателя, помогающего предсказать приток новых клиентов, пользователей в будущем. Часто воспринимается как основной или один из основных показателей отношения клиентов и пользователей к товарам, услугам, брендам. Также используется для оценки своего положения в конкурентном окружении.

Задается всего 1 вопрос о готовности рекомендовать товар/услугу/сервис другим людям, которую просят оценить по 11-ти бальной шкале (от 0 до 10).

Затем оценивших такую готовность от 0 до 6 причисляют к критикам (или детракторам), 7-8 - к нейтралам (или пассивам, т.е. пассивно удовлетворенным), а 9-10 - к сторонникам или промоутерам (т.е. готовым активно рекомендовать).
Затем процент критиков вычитают из процента сторонников. Полученное число считается индексом NPS.

<br>

<center>
<img src="images/nps_calc.png" width="900"> 
</center>

<br>

Эту метрику [считают в куче отраслей.](https://www.retently.com/blog/good-net-promoter-score/)

__Возникает вопрос:__ как для этой метрики считать доверительные интервалы и проверять гипотезы? 

In [1]:
import numpy as np
import scipy.stats as sts

## Доверительный интервал для разности долей

Давайте на примере кликов по дум баннерам выведем формулу для асимптотического доверительного интервала между двумя долями. Представим себе товар, который нужно рекламировать. Для этих целей используется рекламный баннер. Если появляется новый баннер, который кажется более красивым, то возникает необходимость проверить, какой из баннеров лучше, а также понять, насколько новый баннер красивее первого. 

В нашем распоряжении есть выборка из кликов по первому банеру, $X$ и по второму банеру, $Y$.

$$
\begin{aligned}
X_1, \ldots, X_{n_1} \sim iid \hspace{1mm} Bern(p_1) \\
Y_1, \ldots, Y_{n_2} \sim iid \hspace{1mm} Bern(p_2)
\end{aligned}
$$

Мы можем найти оценки для обеих вероятностей через средние. Оба средних будут иметь асимптотически нормальные распределения со своими параметрами: 

$$
\begin{aligned}
\hat p_1 = \bar x \sim N \left(p_1, \frac{p_1 \cdot (1-p_1)}{n_1} \right)\\
\hat p_2 = \bar y \sim N \left(p_2, \frac{p_2 \cdot (1-p_2)}{n_2} \right)
\end{aligned}
$$

Если выборки независимы, разность долей будет тоже иметь асимптотически нормальное распределение: 

$$
\hat p_1 - \hat p_2  \sim N \left(p_1 - p_2, \frac{p_1(1-p_1)}{n_1} + \frac{p_2(1-p_2)}{n_2} \right).
$$

Отсюда можно легко найти доверительный интервал для разности долей: 

$$
\hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}
$$


Применим эту логику к разнице между долей промоутеров и детракторов

> И жёстко облажаемся, потому что выборки в случае NPS зависимы


In [2]:
# -1 детрактор
# 0 промоутер
# 1 нейтрал

choises = np.array([-1, 0, 1]) 
probability = np.array([0.2, 0.5, 0.3])

In [3]:
real_diff = probability[-1] - probability[0]
real_diff

0.09999999999999998

In [4]:
n_sample = 10**3 # размер выборки 
n_obs = 10**5    # число экспериментов

smpl = np.random.choice(choises, p=probability, replace=True, size=(n_obs, n_sample))
smpl

array([[ 1,  0,  0, ...,  1,  1,  1],
       [ 0,  0,  0, ...,  0,  0, -1],
       [ 1,  0,  0, ...,  0, -1,  1],
       ...,
       [ 0, -1,  1, ...,  0,  0,  1],
       [ 0,  1,  0, ...,  1,  1,  0],
       [ 0, -1,  0, ...,  0,  0,  0]])

In [5]:
smpl.shape

(100000, 1000)

In [6]:
def get_norm_ci(sample, alpha):
    """
        Строит нормальный асимптотический 
        доверительный интервал для разности долей 
    """
    
    p_prom = (smpl == 1).mean(axis=1)
    p_detr = (smpl == -1).mean(axis=1)
    
    std_prom = p_prom * (1 - p_prom)/n_sample
    std_detr = p_detr * (1 - p_detr)/n_sample

    diff = p_prom - p_detr
    diff_std = np.sqrt(std_prom + std_detr)
    
    z_alpha = sts.norm().ppf(1 - alpha/2)
    left = diff - z_alpha * diff_std
    right = diff + z_alpha * diff_std
    return left, right

In [7]:
def conf_int_simulation(sample, alpha, get_conf_int, real_diff=real_diff):
    """
        Подсчитываем вероятность попадания в доверительный
        интервал реального NPS
    """
    left, right = get_conf_int(sample, alpha)
    return np.mean((left < real_diff)&(real_diff < right))

In [8]:
conf_int_simulation(smpl, 0.05, get_norm_ci)

0.91315

In [9]:
conf_int_simulation(smpl, 0.01, get_norm_ci)

0.97611

In [10]:
conf_int_simulation(smpl, 0.5, get_norm_ci)

0.44143

Все три эксперемента выдают вероятность покрыть неизвыстный NPS ниже, чем $1 - \alpha$. Это происходит из-за того, что число детракторов коррелирует с числом промоутеров. 

In [11]:
p_prom = (smpl == 1).mean(axis=1)
p_detr = (smpl == -1).mean(axis=1)
np.corrcoef(p_prom, p_detr)[0,1]

-0.32009548962079204

## Доверительный интервал для NPS

Дисперсию надо корректировать на эту корреляцию:

$$
\text{Var}(NPS) = \frac{\hat p_{L} \cdot (1 - \hat p_{L})}{n} + \frac{\hat p_{D} \cdot (1 - \hat p_{D})}{n} + 2 \cdot \frac{\hat p_{L} \cdot \hat p_{D}}{n}.
$$

Здесть $p_L$ и $p_D$ это доли промоутеров и детракторов. Давайте проведём симуляции с новой формулой, а чуть ниже выведем её.

In [12]:
def get_nps_correct_ci(sample, alpha):
    """
        Строит скорректированный интервал 
    """
    P_cnt = (smpl == 1).sum(axis=1)  # число промоутеров
    D_cnt = (smpl == -1).sum(axis=1) # число детракторов
    N_cnt = (smpl == 0).sum(axis=1)  # число нейтралов
    n = smpl.shape[1]                # всего респондентов
    
    NPS_hat = (P_cnt - D_cnt)/n
    
    diff_var = 1/n * (P_cnt/n*(1 - P_cnt/n) + D_cnt/n*(1 - D_cnt/n) + 2* P_cnt/n * D_cnt/n)
    diff_std = np.sqrt(diff_var)

    z_alpha = sts.norm().ppf(1 - alpha/2)
    left = NPS_hat - z_alpha * diff_std
    right = NPS_hat + z_alpha * diff_std
    return left, right

In [13]:
conf_int_simulation(smpl, 0.05, get_nps_correct_ci)    

0.95137

In [14]:
conf_int_simulation(smpl, 0.01, get_nps_correct_ci)    

0.99062

In [15]:
conf_int_simulation(smpl, 0.5, get_nps_correct_ci)    

0.49933

Надёжность доверительных интервалов оказывается на заявленном уровне. Докажем формулу.

Пусть $N$ — число нейтралов, $L$ — число промоутеров, $D$ — число детракторов, $n$ — число наблюдений, $NPS$ вычисляется по формуле 

$$
NPS = \frac{L - D}{n} = \hat p_L - \hat p_D
$$

Значит 

$$
\text{Var}(NPS) = \text{Var} (\hat p_L - \hat p_D) = \text{Var} (\hat p_L) + \text{Var} (\hat p_D) - 2 \cdot \text{Cov}(\hat p_L, \hat p_D) = \frac{p_L (1 - p_L)}{n} + \frac{p_D (1 - p_D)}{n} - 2 \cdot \text{Cov}(\hat p_L, \hat p_D).
$$

Найдём ковариацию

$$
\text{Cov}(\hat p_L, \hat p_D) = \text{Cov}\left( \frac{L}{n}, \frac{D}{n} \right) = \frac{1}{n^2}\text{Cov}\left(L, D \right)
$$

Воспользуемся для поиска ковариации тем, что $N + P + D = n$

$$
\text{Cov}(L + N + D, L) = 0 \Rightarrow \text{Var}(L) + \text{Cov}(L, N) + \text{Cov}(L, D) = 0
$$

По аналогии можно выписать уравнение для $D$ и $N$, получится система уравнений

\begin{equation*}
    \begin{cases}
      \text{Var}(L) + \text{Cov}(L, N) + \text{Cov}(L, D) = 0 \\
      \text{Var}(D) + \text{Cov}(N, D) + \text{Cov}(L, D) = 0 \\
      \text{Var}(N) + \text{Cov}(L, N) + \text{Cov}(N, D) = 0 \\
    \end{cases}
\end{equation*}

Решаем её и получаем

\begin{equation*}
    \begin{aligned}
        & \text{Cov}(L, N) = \frac{-\text{Var}(L) + \text{Var}(D) - \text{Var}(N)}{2} \\
        & \text{Cov}(N, D) = \frac{\text{Var}(L) - \text{Var}(D) - \text{Var}(N) }{2} \\
        & \text{Cov}(L, D) = \frac{-\text{Var}(L) - \text{Var}(D) + \text{Var}(N) }{2} \\
    \end{aligned}
\end{equation*}

Подставим значение дисперсий

$$
\text{Cov}(L, D) = \frac{-\text{Var}(L) - \text{Var}(D) + \text{Var}(N) }{2} = \frac{n}{2} \cdot (- p_D \cdot (1 -  p_D) + p_N \cdot (1 -  p_N) - p_L \cdot (1 - p_L))
$$

Вспомним, что $p_N = 1 - p_L -  p_D$ и раскроем скобки, получим

$$
\text{Cov}(L, D) = -n \cdot p_L \cdot p_D.
$$

Подставим в формулу дисперсии

$$
\text{Var}(NPS) = \frac{p_L (1 - p_L)}{n} + \frac{p_D (1 - p_D)}{n} - 2 \cdot \frac{-n \cdot p_L \cdot p_D}{n^2}
$$

Заменим вероятности на их оценки и получим требуемую формулу

$$
\text{Var}(NPS) = \frac{\hat p_L (1 - \hat p_L)}{n} + \frac{\hat p_D (1 - \hat p_D)}{n} + \frac{2 \cdot  \hat p_L \cdot \hat p_D}{n}
$$